# results_preliminary_for_hlca

In [1]:
1

1

In [2]:
import os
import scanpy as sc
import pickle
import sys
from collections import Counter
sys.path.append("../..")

In [3]:
from idtrack import *
from idtrack._track_tests import *
logger_config()
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"  # or any other local directory
dm = DatabaseManager("homo_sapiens", 107, "gene", local_dir, 106) 
tt: TrackTests = TrackTests(dm)


In [6]:
base_path = "/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data"
dset0_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/ready/full")
dset1_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/raw")

adata_dict = {
    "Kaminski_2020": [f"{dset0_dir}/adams.h5ad"],
    "Meyer_2021": [f"{dset0_dir}/meyer_2021.h5ad"],
    "MeyerNikolic_unpubl": [f"{dset0_dir}/meyer_nikolic_unpubl.h5ad"],
    "Barbry_unpubl": [f"{dset0_dir}/barbry.h5ad"],
    "Regev_2021": [
        f"{dset0_dir}/delorey_cryo.h5ad", f"{dset0_dir}/delorey_fresh.h5ad",
        f"{dset0_dir}/delorey_nuclei.h5ad"
    ],
    "Thienpont_2018": [f"{dset1_dir}/Lambrechts/lambrechts.h5ad"],
    "Budinger_2020": [f"{dset0_dir}/bharat.h5ad"],
    "Banovich_Kropski_2020": [f"{dset0_dir}/haberman.h5ad"],
    "Sheppard_2020": [f"{dset0_dir}/tsukui.h5ad"],
    "Wunderink_2021": [
        f"{dset0_dir}/grant_cryo.h5ad", f"{dset0_dir}/grant_fresh.h5ad"
    ],
    "Lambrechts_2021": [
        f"{dset0_dir}/wouters.h5ad" #, f"{dset0_dir}/wouters_labs.h5ad"
    ],
    "Zhang_2021": [f"{dset1_dir}/Liao/covid_for_publish.h5ad"],
    "Duong_lungMAP_unpubl": [f"{dset0_dir}/duong.h5ad"],
    "Janssen_2020": [f"{dset0_dir}/mould.h5ad"],
    "Sun_2020": [
        f"{dset0_dir}/wang_sub_batch1.h5ad", f"{dset0_dir}/wang_sub_batch2.h5ad",
        f"{dset0_dir}/wang_sub_batch3.h5ad", f"{dset0_dir}/wang_sub_batch4.h5ad"],
    "Gomperts_2021": [
        f"{dset0_dir}/carraro_ucla.h5ad", f"{dset0_dir}/carraro_cff.h5ad",
        f"{dset0_dir}/carraro_csmc.h5ad"],
    "Eils_2020": [f"{dset0_dir}/lukassen.h5ad"],
    "Schiller_2020": [f"{dset0_dir}/mayr.h5ad"],
    "Misharin_Budinger_2018": [f"{dset0_dir}/reyfman_disease.h5ad"],
    "Shalek_2018": [f"{dset0_dir}/ordovasmontanes.h5ad"],
    "Schiller_2021": [f"{dset0_dir}/schiller_discovair.h5ad"],
    "Peer_Massague_2020": [f"{dset0_dir}/laughney.h5ad"],
    "Lafyatis_2019": [f"{dset0_dir}/valenzi.h5ad"],
    "Tata_unpubl": [f"{dset0_dir}/tata_unpubl.h5ad"],
    "Xu_2020": [f"{dset0_dir}/guo.h5ad"],
    "Sims_2019": [f"{dset0_dir}/szabo.h5ad"],
    "Schultze_unpubl": [f"{dset0_dir}/schultze_unpubl.h5ad"]
}

##### Conversion and save

In [ ]:
results = dict()

for ddadaata in adata_dict:
    
    adata = sc.read(adata_dict[ddadaata][0])
    gene_list = list(adata.var.index)
    
    entered_gene_list = list()
    return_list = list()
    
    print(ddadaata)
    for ind, gl in enumerate(gene_list):
        if ind % 100 == 0 or ind > len(gene_list)-5:
            progress_bar(ind, len(gene_list)-1)
        
        new_gl, is_converted = tt.unfound_node_solutions(gl)
        if new_gl is None:
            new_gl, is_converted = tt.unfound_node_solutions(f"{DB.synonym_id_nodes_prefix}{gl}")
            is_converted = True if new_gl is not None else False

        if new_gl is None:
            return_list.append([])
        elif new_gl and is_converted:
            conv = tt.convert(new_gl, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=False)
            return_list.append(list(conv) if conv is not None else [])
            entered_gene_list.append(new_gl)
        elif new_gl:
            conv = tt.convert(gl, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=False)
            return_list.append(list(conv) if conv is not None else [])
            entered_gene_list.append(gl)
        else:
            raise ValueError
    print(tt.identify_source(entered_gene_list)[0])
    print(list(Counter([len(i) for i in return_list]).most_common())[:10])
    assert len(gene_list) == len(return_list)
    results[ddadaata] = {i: j for i, j in zip(gene_list, return_list)}
    print()

file_path = os.path.join(local_dir, "results_preliminary_for_hlca.pk")
with open(file_path, 'wb') as handle:
    pickle.dump(result, handle)
    print(f"Saved: {file_path}")

In [7]:
file_path = os.path.join(local_dir, "results_preliminary_for_hlca.pk")
with open(file_path, 'wb') as handle:
    pickle.dump(results, handle)
    print(f"Saved: {file_path}")

Saved: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/results_preliminary_for_hlca.pk


In [4]:
file_path = os.path.join(local_dir, "results_preliminary_for_hlca.pk")
with open(file_path, 'rb') as handle:
    results = pickle.load(handle)
    print(f"Loaded: {file_path}")

Loaded: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/results_preliminary_for_hlca.pk


In [23]:
adata.var['gene_ids']

TSPAN6           ENSG00000000003
TNMD             ENSG00000000005
DPM1             ENSG00000000419
SCYL3            ENSG00000000457
C1orf112         ENSG00000000460
                      ...       
RP11-11M20.4     ENSG00000283078
LLNLR-245B6.1    ENSG00000283103
CTD-2060L22.1    ENSG00000283117
RP11-107E5.4     ENSG00000283118
RP11-299P2.2     ENSG00000283125
Name: gene_ids, Length: 28527, dtype: object

##### all

In [6]:
adata_dict = {"Additional_dataset": ["/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data/HLCA_core_h5ads/HLCA_v2.h5ad"]}
results = dict()
for ddadaata in adata_dict:
    
    adata = sc.read(adata_dict[ddadaata][0])
    gene_list = list(adata.var.index)
    
    entered_gene_list = list()
    return_list = list()
    
    print(ddadaata)
    for ind, gl in enumerate(gene_list):
        if ind % 100 == 0 or ind > len(gene_list)-5:
            progress_bar(ind, len(gene_list)-1)
        
        new_gl, is_converted = tt.unfound_node_solutions(gl)
        if new_gl is None:
            new_gl, is_converted = tt.unfound_node_solutions(f"{DB.synonym_id_nodes_prefix}{gl}")
            is_converted = True if new_gl is not None else False

        if new_gl is None:
            return_list.append([])
        elif new_gl and is_converted:
            conv = tt.convert(new_gl, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=False)
            return_list.append(list(conv) if conv is not None else [])
            entered_gene_list.append(new_gl)
        elif new_gl:
            conv = tt.convert(gl, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=False)
            return_list.append(list(conv) if conv is not None else [])
            entered_gene_list.append(gl)
        else:
            raise ValueError
    print(tt.identify_source(entered_gene_list)[0])
    print(list(Counter([len(i) for i in return_list]).most_common())[:10])
    assert len(gene_list) == len(return_list)
    results[ddadaata] = {i: j for i, j in zip(gene_list, return_list)}
    print()

file_path = os.path.join(local_dir, "results_preliminary_for_hlca_V2_ALL.pk")
with open(file_path, 'wb') as handle:
    pickle.dump(results, handle)
    print(f"Saved: {file_path}")

Additional_dataset
Progress: |████████████████████| 100.0% 
(('Vega gene', 88), 26626)
[(1, 28173), (0, 352), (3, 1), (2, 1)]

Saved: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/results_preliminary_for_hlca_V2_ALL.pk


In [7]:
file_path = os.path.join(local_dir, "results_preliminary_for_hlca_V2_ALL.pk")
with open(file_path, 'rb') as handle:
    results = pickle.load(handle)
    print(f"Loaded: {file_path}")

Loaded: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/results_preliminary_for_hlca_V2_ALL.pk


##### how many target hit

In [9]:
all_targets = {j for database in results for i in results[database] for j in results[database][i]}
len(all_targets)

27911

In [10]:
exts = dm.change_release(107).get_db('external_relevant')

In [11]:
exts=exts[exts['name_db'] == 'HGNC Symbol']

In [12]:
df_ciro = exts[exts["graph_id"].isin(all_targets)]

In [13]:
df_ciro

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity
30645,107,ENSG00000000003.15,TSPAN6,HGNC Symbol,NaN,NaN
30646,107,ENSG00000000005.6,TNMD,HGNC Symbol,NaN,NaN
30647,107,ENSG00000000419.14,DPM1,HGNC Symbol,NaN,NaN
30648,107,ENSG00000000457.14,SCYL3,HGNC Symbol,NaN,NaN
30649,107,ENSG00000000460.17,C1orf112,HGNC Symbol,NaN,NaN
...,...,...,...,...,...,...
75384,107,ENSG00000288702.1,UGT1A3,HGNC Symbol,NaN,NaN
75386,107,ENSG00000288709.1,F8A2,HGNC Symbol,NaN,NaN
75389,107,ENSG00000288825.1,H2AC18,HGNC Symbol,NaN,NaN
75397,107,ENSG00000289604.1,CTAGE8,HGNC Symbol,NaN,NaN


In [15]:
import pandas as pd

for i in range(79, 108):
    print(i, end=', ')
    exts_ = dm.change_release(i).get_db('external_relevant')
    exts_=exts_[exts_['name_db'] == 'HGNC Symbol']
    exts=pd.concat([exts, exts_])

79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 

In [16]:
df_ciro_u = exts[exts["graph_id"].isin(all_targets)]
df_ciro_u = df_ciro_u.drop_duplicates(["graph_id", "id_db"])
df_ciro_u.shape

(22456, 6)

In [69]:
df_ciro_u[df_ciro_u['id_db'] == "CKMT1A"]

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity


In [70]:
df_ciro[df_ciro['id_db'] == "CKMT1A"]

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity


In [56]:
for i in range(79, 108):
    print(i, end=', ')
    exts_ = dm.change_release(i).get_db('external_relevant')
    exts_=exts_[exts_['name_db'] == 'CKMT1A']
    if len(exts_) > 0:
        print(i)
        break
#    exts=pd.concat([exts, exts_])

79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 

In [55]:
len(df_ciro_u)

40902

In [66]:
exts[exts["graph_id"] == "ENSG00000223572.10"]

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity
53910,107,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN
70502,100,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN
75770,101,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN
76869,102,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN
74978,103,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN
53185,104,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN
53661,105,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN
50535,106,ENSG00000223572.10,CKMT1A,HGNC Symbol,NaN,NaN


In [101]:
tt.convert("OPN3", None, None, 'ensembl_gene', prioritize_to_one_filter=False, return_path=True)

{'ENSG00000203668.3': [1,
  1,
  nan,
  0,
  (('OPN3', 'ENSG00000203668.3', 0, ('ensembl_gene', 102)),)],
 'ENSG00000054277.14': [1,
  1,
  nan,
  0,
  (('OPN3', 'ENSG00000054277.14', 0, ('ensembl_gene', 96)),)]}

In [100]:
tt.convert("CKMT1B", None, None, 'ensembl_gene', prioritize_to_one_filter=False, return_path=True)

{'ENSG00000237289.10': [1,
  1,
  nan,
  0,
  (('CKMT1B', 'ENSG00000237289.10', 0, ('ensembl_gene', 100)),)],
 'ENSG00000223572.10': [1,
  3,
  nan,
  0,
  (('CKMT1B', 'ENSG00000223572.5', 0, ('assembly_37_ensembl_gene', 100)),
   ('ENSG00000223572.5', '548596', 0, ('external', 100)),
   ('548596', 'ENSG00000223572.10', 0, ('ensembl_gene', 100)))]}

In [82]:
list(tt.reverse_graph.neighbors("ENSG00000223572.5"))

['1159',
 '548596',
 'CKMT1A',
 'CKMT1B',
 'OTTHUMG00000059898',
 'ENST00000434505.1',
 'ENST00000411750.1',
 'ENST00000413453.2',
 'ENST00000415044.1']

In [85]:
tt.graph["ENSG00000223572"]

AdjacencyView({'ENSG00000223572.8': {0: {'releases': {80, 79}}}, 'ENSG00000223572.9': {0: {'releases': {81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}}}, 'ENSG00000223572.10': {0: {'releases': {100, 101, 102, 103, 104, 105, 106, 107}}}})

In [94]:
['CKMT1A' in results[i].keys() for i in results]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False]

In [20]:
df_ciro_u[['graph_id', 'id_db']].to_csv("/lustre/groups/ml01/workspace/kemal.inecik/hlca_id_mapping_preliminary/results_preliminary_for_hlca_gene_name_ensembl_matching_V2_ALL.csv", 
               index=False)

In [21]:
pd.read_csv("/lustre/groups/ml01/workspace/kemal.inecik/hlca_id_mapping_preliminary/results_preliminary_for_hlca_gene_name_ensembl_matching_V2_ALL.csv")

,graph_id,id_db
0,ENSG00000000003.15,TSPAN6
1,ENSG00000000005.6,TNMD
2,ENSG00000000419.14,DPM1
3,ENSG00000000457.14,SCYL3
4,ENSG00000000460.17,C1orf112
...,...,...
22451,ENSG00000233058.3,LINC00884
22452,ENSG00000233621.3,LINC01137
22453,ENSG00000235703.7,LINC00894
22454,ENSG00000167085.14,PHB


In [ ]:
neighbors

In [27]:
'CHML' in list(df_ciro_u['id_db'])

True

In [11]:
'MT-CO1' in list(df_ciro['id_db'])

True

In [21]:
'COX1' in list(df_ciro['id_db'])

False

In [23]:
df_ciro.to_pickle(os.path.join(local_dir, "results_preliminary_for_hlca_gene_name_ensembl_matching.pk"))
os.path.join(local_dir, "results_preliminary_for_hlca_gene_name_ensembl_matching.pk")

'/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/results_preliminary_for_hlca_gene_name_ensembl_matching.pk'

In [30]:
df_ciro[['graph_id', 'id_db']].to_csv("/lustre/groups/ml01/workspace/kemal.inecik/hlca_id_mapping_preliminary/results_preliminary_for_hlca_gene_name_ensembl_matching.csv", 
               index=False)

In [31]:
pd.read_csv("/lustre/groups/ml01/workspace/kemal.inecik/hlca_id_mapping_preliminary/results_preliminary_for_hlca_gene_name_ensembl_matching.csv")

,graph_id,id_db
0,ENSG00000000003.15,TSPAN6
1,ENSG00000000005.6,TNMD
2,ENSG00000000419.14,DPM1
3,ENSG00000000457.14,SCYL3
4,ENSG00000000460.17,C1orf112
...,...,...
39735,ENSG00000289549.1,PPIAL4D
39736,ENSG00000289604.1,CTAGE8
39737,ENSG00000289731.1,FAM153B
39738,ENSG00000289811.1,FAM95A


In [20]:
import pandas as pd
pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/hlca_id_mapping_preliminary/results_preliminary_for_hlca_gene_name_ensembl_matching.pk")

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity
30645,107,ENSG00000000003.15,TSPAN6,HGNC Symbol,NaN,NaN
30646,107,ENSG00000000005.6,TNMD,HGNC Symbol,NaN,NaN
30647,107,ENSG00000000419.14,DPM1,HGNC Symbol,NaN,NaN
30648,107,ENSG00000000457.14,SCYL3,HGNC Symbol,NaN,NaN
30649,107,ENSG00000000460.17,C1orf112,HGNC Symbol,NaN,NaN
...,...,...,...,...,...,...
75396,107,ENSG00000289549.1,PPIAL4D,HGNC Symbol,NaN,NaN
75397,107,ENSG00000289604.1,CTAGE8,HGNC Symbol,NaN,NaN
75401,107,ENSG00000289731.1,FAM153B,HGNC Symbol,NaN,NaN
75403,107,ENSG00000289811.1,FAM95A,HGNC Symbol,NaN,NaN


In [8]:
for database in results:
    all_targets = Counter(j for i in results[database] for j in results[database][i])
    n_to_1_events = [j for i in results[database] for j in results[database][i] if all_targets[j] != 1]
    one_to_events = [i for i in results[database] if len(results[database][i]) > 1]
    one_to_0_events = [i for i in results[database] if len(results[database][i]) < 1]
    print(f"{database}\nSource_Count:{len(results[database])}\nTarget_Count:{len(all_targets)}")
    print(f"n-to-1_events:{len(n_to_1_events)}\n1-to-n_events:{len(one_to_events)}\n1-to-0_events:{len(one_to_0_events)}")
    print()

Kaminski_2020
Source_Count:45947
Target_Count:45378
n-to-1_events:305
1-to-n_events:0
1-to-0_events:411

Meyer_2021
Source_Count:20922
Target_Count:20801
n-to-1_events:189
1-to-n_events:0
1-to-0_events:24

MeyerNikolic_unpubl
Source_Count:33582
Target_Count:32815
n-to-1_events:1145
1-to-n_events:4
1-to-0_events:174

Barbry_unpubl
Source_Count:16859
Target_Count:16768
n-to-1_events:126
1-to-n_events:1
1-to-0_events:29

Regev_2021
Source_Count:30983
Target_Count:30388
n-to-1_events:826
1-to-n_events:1
1-to-0_events:173

Thienpont_2018
Source_Count:27958
Target_Count:27428
n-to-1_events:393
1-to-n_events:1
1-to-0_events:327

Budinger_2020
Source_Count:26316
Target_Count:25943
n-to-1_events:545
1-to-n_events:1
1-to-0_events:96

Banovich_Kropski_2020
Source_Count:33694
Target_Count:32824
n-to-1_events:731
1-to-n_events:6
1-to-0_events:497

Sheppard_2020
Source_Count:27147
Target_Count:26617
n-to-1_events:409
1-to-n_events:1
1-to-0_events:320

Wunderink_2021
Source_Count:21819
Target_Count:2

In [31]:
ens_ids_107 = tt.get_id_list('ensembl_gene', 107)
ens_ids_107 = set(ens_ids_107)
problematic_ids = {i for i in all_targets if i not in ens_ids_107}
problematic_ids  # empty if all are from the ens_ids_107

set()

In [32]:
count = 0
for i in all_targets:
    if len(tt.synonymous_nodes(i, 2, {"HGNC Symbol"}, from_release=107)) == 0:
        count += 1

In [33]:
count, len(all_targets)

(17428, 57606)

##### HGNC vs Ensembl gene as targets

In [ ]:
ddadaata = "Kaminski_2020"
adata = sc.read(adata_dict[ddadaata][0])

gene_list = list(adata.var.index)
entered_gene_list = list()
return_list = list()
print(ddadaata, len(gene_list))

for ind, gl in enumerate(gene_list):
    
    if ind % 100 == 0 or ind > len(gene_list)-5:
        progress_bar(ind, len(gene_list)-1)
    
    new_gl, is_converted = tt.unfound_node_solutions(gl)
    if new_gl is None:
        new_gl, is_converted = tt.unfound_node_solutions(f"{DB.synonym_id_nodes_prefix}{gl}")
        is_converted = True if new_gl is not None else False

    if new_gl is None:
        return_list.append([])
    elif new_gl and is_converted:
        conv = tt.convert(new_gl, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=False)
        return_list.append(list(conv) if conv is not None else [])
        entered_gene_list.append(new_gl)
    elif new_gl:
        conv = tt.convert(gl, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=False)
        return_list.append(list(conv) if conv is not None else [])
        entered_gene_list.append(gl)
    else:
        raise ValueError
print(tt.identify_source(entered_gene_list)[0])
print(list(Counter([len(i) for i in return_list]).most_common())[:10])
print()

gene_list = list(adata.var.index)
entered_gene_list = list()
return_list = list()
print(ddadaata, len(gene_list))

for ind, gl in enumerate(gene_list):
    if ind % 100 == 0 or ind > len(gene_list)-5:
        progress_bar(ind, len(gene_list)-1)

    new_gl, is_converted = tt.unfound_node_solutions(gl)
    if new_gl is None:
        new_gl, is_converted = tt.unfound_node_solutions(f"{DB.synonym_id_nodes_prefix}{gl}")
        is_converted = True if new_gl is not None else False

    if new_gl is None:
        return_list.append([])
    elif new_gl and is_converted:
        conv = tt.convert(new_gl, None, None, 'HGNC Symbol', prioritize_to_one_filter=True, return_path=False)
        return_list.append(list(conv) if conv is not None else [])
        entered_gene_list.append(new_gl)
    elif new_gl:
        conv = tt.convert(gl, None, None, 'HGNC Symbol', prioritize_to_one_filter=True, return_path=False)
        return_list.append(list(conv) if conv is not None else [])
        entered_gene_list.append(gl)
    else:
        raise ValueError
print(tt.identify_source(entered_gene_list)[0])
print(list(Counter([len(i) for i in return_list]).most_common())[:10])
print()

In [ ]:
[gene_list[ind] for ind, i in enumerate(return_list) if len(i) == 0][:10]